In [1]:
import pandas as pd
from elasticsearch import Elasticsearch
es = Elasticsearch('http://localhost:9200/')
es.ping()

True

In [6]:
data = pd.read_csv("reviews.csv")
data.head()

,business_name,author_name,text,photo,rating,rating_category
0,Haci'nin Yeri - Yigit Lokantasi,Gulsum Akar,We went to Marmaris with my wife for a holiday...,dataset/taste/hacinin_yeri_gulsum_akar.png,5,taste
1,Haci'nin Yeri - Yigit Lokantasi,Oguzhan Cetin,During my holiday in Marmaris we ate here to f...,dataset/menu/hacinin_yeri_oguzhan_cetin.png,4,menu
2,Haci'nin Yeri - Yigit Lokantasi,Yasin Kuyu,Prices are very affordable. The menu in the ph...,dataset/outdoor_atmosphere/hacinin_yeri_yasin_...,3,outdoor_atmosphere
3,Haci'nin Yeri - Yigit Lokantasi,Orhan Kapu,Turkey's cheapest artisan restaurant and its f...,dataset/indoor_atmosphere/hacinin_yeri_orhan_k...,5,indoor_atmosphere
4,Haci'nin Yeri - Yigit Lokantasi,Ozgur Sati,I don't know what you will look for in terms o...,dataset/menu/hacinin_yeri_ozgur_sati.png,3,menu


In [7]:
df = data.drop(columns=['photo', 'author_name'], axis=1)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1100 entries, 0 to 1099
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   business_name    1100 non-null   object
 1   text             1100 non-null   object
 2   rating           1100 non-null   int64 
 3   rating_category  1100 non-null   object
dtypes: int64(1), object(3)
memory usage: 34.5+ KB


In [9]:
df = df.rename(columns={'business_name': 'restaurant_name', 'text': 'review'})

In [ ]:
import os
API_KEY = os.environ['GEMINI_API_KEY']

In [10]:
import google.generativeai as genai
genai.configure(api_key=API_KEY)

In [5]:
result = genai.embed_content(model="models/text-embedding-004", content="flower")
print(str(result["embedding"]))

[0.011361456, 0.027189797, -0.025959939, 0.011108013, 0.03285467, 0.05067413, 0.0011572732, -0.02865956, 0.10035564, 0.030404288, -0.029409122, 0.040363513, 0.020189904, -0.0015644672, -0.0575784, -0.09468042, 0.031010356, 0.035894867, -0.025067603, 0.009278434, 0.10849758, -0.012063715, 0.03284496, 0.01984334, -0.04848009, -0.011043656, -0.015488618, 0.080078766, 0.029904012, -0.072724156, -0.021997293, 0.048937634, 0.08026301, 0.034169752, 0.010945666, 0.021574749, -0.026549382, 0.06324018, -0.004253098, -0.020592606, -0.041103072, 0.05100106, -0.033422455, 0.026154175, -0.025389679, -0.018875165, 0.05056579, 0.017226197, 0.0018336317, 0.043104388, -0.008480716, 0.034568373, -0.01091496, 0.020113599, 0.0035839994, -0.002407264, -0.05059118, 0.01432583, 0.086445145, 0.064168595, -0.0575413, -0.003634417, -0.017181475, -0.029051695, 0.022423983, 0.049755197, -0.07406149, 0.036045685, -0.036256343, 0.007444632, 0.0045666904, 0.05550775, -0.01633727, 0.02643737, -0.005407431, 0.007064586

In [11]:
def create_embedding(text):
    result = genai.embed_content(model="models/text-embedding-004", content=text)
    return result['embedding']

In [12]:
df['review_vector'] = df['review'].apply(lambda x: create_embedding(x))

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1100 entries, 0 to 1099
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   restaurant_name  1100 non-null   object
 1   review           1100 non-null   object
 2   rating           1100 non-null   int64 
 3   rating_category  1100 non-null   object
 4   review_vector    1100 non-null   object
dtypes: int64(1), object(4)
memory usage: 43.1+ KB


In [15]:
indexMapping = {
    "properties":{
        "restaurant_name":{
            "type":"text"
        },
        "review":{
            "type":"text"
        },
        "rating":{
            "type":"integer"
        },
        "rating_category":{
            "type":"text"
        },
        "review_vector":{
            "type":"dense_vector",
            "dims": 768,
            "index" :True,
            "similarity": "cosine"
        }
    }
}
es.indices.create(index="review_index", mappings=indexMapping)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'review_index'})

In [17]:
i=0
records = df.to_dict("records")
for record in records:
    es.index(
    index="review_index",
    id=i,
    document=record)
    i+=1

In [18]:
res = es.get(index="review_index", id=2)
res['_source']

{'restaurant_name': "Haci'nin Yeri - Yigit Lokantasi",
 'review': 'Prices are very affordable. The menu in the photo cost 108 liras. You have to wait 10-15 minutes for food. Staff is annoying. Well it tastes good. Boiled meat was delicious.',
 'rating': 3,
 'rating_category': 'outdoor_atmosphere',
 'review_vector': [0.0022617013,
  -0.04092595,
  0.014304046,
  -0.0024255172,
  -0.005678478,
  -0.025250072,
  -0.051936258,
  0.0069213696,
  -0.020835586,
  0.011741798,
  0.08145663,
  0.06709783,
  0.07938421,
  -0.019769568,
  0.013476992,
  -0.0737847,
  0.022880813,
  0.023685478,
  -0.07027303,
  -0.05996867,
  0.023060786,
  -0.014928286,
  0.024597224,
  -0.045226842,
  -0.047589213,
  -0.06037469,
  -0.0450711,
  0.01556512,
  0.030881891,
  0.015204088,
  0.056134507,
  0.026337974,
  -0.010314166,
  -0.007494954,
  0.051289227,
  0.017380957,
  -0.022891238,
  0.014975023,
  0.021961588,
  -0.036231596,
  -0.045639925,
  0.062233485,
  0.032322317,
  -0.014286372,
  0.00289142

In [19]:
queryInput = "party place"
vector = create_embedding(queryInput)

knn = {
    'field': "review_vector",
    'query_vector': vector,
    'k':10
}

res = es.search(index='review_index', knn=knn, source=['restaurant_name','review'])
res['hits']['hits']

[{'_index': 'review_index',
  '_id': '898',
  '_score': 0.79406834,
  '_source': {'restaurant_name': 'Karakoy Lokantasi',
   'review': 'Awesome place!'}},
 {'_index': 'review_index',
  '_id': '115',
  '_score': 0.7849543,
  '_source': {'restaurant_name': 'Pukka Marmaris',
   'review': 'Wonderful place where you can go with your family and have fun; nice place with live music...'}},
 {'_index': 'review_index',
  '_id': '473',
  '_score': 0.78436375,
  '_source': {'restaurant_name': 'Zula',
   'review': "Surprised location; but nice clean venue. The employees are friendly; I don't want to comment on the prices; they were too high.."}},
 {'_index': 'review_index',
  '_id': '1045',
  '_score': 0.7833431,
  '_source': {'restaurant_name': 'Lipa',
   'review': 'Great venue great food appetizers. I can be a customer from now on.'}},
 {'_index': 'review_index',
  '_id': '604',
  '_score': 0.7819269,
  '_source': {'restaurant_name': 'KFC',
   'review': 'The place is very nice; I think, the food 

In [20]:
# res['hits']['hits'][0]['_source']['review']
documents = []
for doc in res['hits']['hits']:
    documents.append(str(doc['_source']))
print(documents)

["{'restaurant_name': 'Karakoy Lokantasi', 'review': 'Awesome place!'}", "{'restaurant_name': 'Pukka Marmaris', 'review': 'Wonderful place where you can go with your family and have fun; nice place with live music...'}", '{\'restaurant_name\': \'Zula\', \'review\': "Surprised location; but nice clean venue. The employees are friendly; I don\'t want to comment on the prices; they were too high.."}', "{'restaurant_name': 'Lipa', 'review': 'Great venue great food appetizers. I can be a customer from now on.'}", "{'restaurant_name': 'KFC', 'review': 'The place is very nice; I think, the food was fresh and good; one of my favorite places.'}", "{'restaurant_name': 'Leman Kultur', 'review': 'A large spacious place. If you want; you can relax inside and have something to drink.'}", "{'restaurant_name': 'Lipa', 'review': 'Great and chill place; ambiance was nice too.'}", "{'restaurant_name': 'Ege Vera', 'review': 'A delicious and clean place.'}", "{'restaurant_name': 'Pecko Firin Atakum', 'revi

In [21]:
query = "where can i have fun"

In [22]:
output = "output in json format. give top suggestions and final conclusions"

In [23]:
model = genai.GenerativeModel("gemini-1.5-flash")
prompt = f"You are a cool resturant suggestor. Answer the question: '{query}'.\n\nContext:\n{''.join(documents)} and also show the most relevent review text. output: {output}"
response = model.generate_content(prompt)
print(response.text) 

```json
{
  "suggestions": [
    {
      "restaurant_name": "Pukka Marmaris",
      "relevance_score": 1.0,
      "review_snippet": "Wonderful place where you can go with your family and have fun; nice place with live music...",
      "reason": "This restaurant explicitly mentions 'have fun' and live music, suggesting a lively and entertaining atmosphere."
    },
    {
      "restaurant_name": "Karakoy Lokantasi",
      "relevance_score": 0.8,
      "review_snippet": "Awesome place!",
      "reason": "While not explicitly mentioning 'fun', the enthusiastic 'Awesome place!' suggests a positive and potentially enjoyable experience."
    },
    {
      "restaurant_name": "Lipa",
      "relevance_score": 0.7,
      "review_snippet": "Great and chill place; ambiance was nice too.",
      "reason": "'Chill' implies a relaxed and enjoyable atmosphere, which can be a form of fun for some."
    },
    {
      "restaurant_name": "Kumsal Lokantasi",
      "relevance_score": 0.7,
      "review_sni